In [5]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Był problem z długością ścieżki stąd wymuszam ukrórcenie poprzez instalacje modelu w folderze bezpośrednio na dysku C
custom_cache = "C:/hf_cache"
if not os.path.exists(custom_cache):
    os.makedirs(custom_cache)

df = pd.read_csv('../Data/NPS_2025.csv', sep=';', keep_default_na=False, na_values=[''], low_memory=False)

# NPS 7 lub 8
subset = df[
    (df['NPS & OSAT comment'].notna()) & 
    (df['NPS & OSAT comment'] != '') &
    (df['NPS & OSAT comment'] != 'N/A') &
    (df['NPS'].isin([7, 8]))
][['NPS', 'NPS & OSAT comment']].copy()

print(f"Liczba komentarzy do analizy (niepuste, NPS 7-8): {len(subset)}")

# ładowanie modelu XLM-RoBERTa
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=custom_cache, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir=custom_cache)
    
    # Zamiast na CPU tworzę pipeline na GPU aby było szybciej
    classifier = pipeline(
        "sentiment-analysis", 
        model=model, 
        tokenizer=tokenizer, 
        device = 0,  # W przypadku braku GPU proszę zamienić na -1, czyli CPU
        truncation=True, 
        max_length=512
    )
    print("Model załadowany")

except Exception as e:
    print(f"Błąd ładowania modelu: {e}")
    # W przypadku błędu związanego z nieutworzeniem folderu na dysku C zatrzymuję kod już w tym momencie
    raise e
    
# Przygotowanie danych (konwersja kolumny na listę)
# Zamiana na listę stringów jest konieczna dla pipeline'u
comments_list = subset['NPS & OSAT comment'].astype(str).tolist()

results = []
batch_size = 16
print(f"Rozpoczynam analizę {len(comments_list)} komentarzy...")
# Iteracja po liście z krokiem 'batch_size'
for i in range(0, len(comments_list), batch_size):
    # Wybór tekstów
    batch = comments_list[i : i + batch_size]
    # Model przetwarza całą paczkę na raz
    predictions = classifier(batch)
    # Zapis wyników z tej paczki
    for out in predictions:
        label = out['label']
        if label == 'negative': 
            results.append('Negative')
        else:
            results.append('Neutral_or_Positive')
            
# Przypisanie wyników z powrotem do DataFrame
subset['Sentiment_Flag'] = results

# Zapis pliku do folderu Data
output_path = '../Data/nps_sentiment_results.csv'
subset[['Sentiment_Flag']].to_csv(output_path, index=True)

print(f"Gotowe! Wyniki zapisano do: {output_path}")
print(subset[['NPS & OSAT comment', 'Sentiment_Flag']].head())

Liczba komentarzy do analizy (niepuste, NPS 7-8): 9208
Model załadowany


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Gotowe! Wyniki zapisano do: ../Data/nps_sentiment_results.csv
                                   NPS & OSAT comment       Sentiment_Flag
0                                          Easy going  Neutral_or_Positive
1   Not sure what was going on but it took over an...  Neutral_or_Positive
3                                      Cena za wysoka  Neutral_or_Positive
9                                Opóźnienie w wylocie  Neutral_or_Positive
11                              Good reliable on time  Neutral_or_Positive


In [6]:
print(subset['Sentiment_Flag'].value_counts())

Sentiment_Flag
Neutral_or_Positive    5200
Negative               4008
Name: count, dtype: int64


In [7]:
# Filtrowanie i wyświetlenie pierwszych 10 wyników
negative_subset = subset[subset['Sentiment_Flag'] == 'Negative']
print(negative_subset.head(10))

     NPS                                 NPS & OSAT comment Sentiment_Flag
19     7                    Too many low charter equipment.       Negative
35     7  Nice flight. However old plane so impossible t...       Negative
69     7  Jakaś stara baba zajęła moje miejsce i na kilk...       Negative
93     8  I had a missed connection twice within 3  week...       Negative
101    7                    Not a nice boarding experience.       Negative
108    7  Inconvenience about PRG-WAW. There was 90 min ...       Negative
135    8            wszystko OK, minus za lekkie spóźnienie       Negative
146    7        Slightly delaying on flights but managable.       Negative
187    7       Delay without, apparently, a serious reason.       Negative
191    7  Not ok, because When I did the check in I coul...       Negative
